# Introduction

This code shows how to use Azure Machine Learning (AML) managed online endpoints with PyRIT.

## Prerequisites

1. **Deploy an AML-Managed Online Endpoint:** Confirm that an Azure Machine Learning managed online endpoint is
     already deployed.

1. **Obtain the API Key:**
   - Navigate to the AML Studio.
   - Go to the 'Endpoints' section.
   - Retrieve the API key and endpoint URI.
   <br> <img src="./../../assets/aml_managed_online_endpoint_api_key.png" alt="aml_managed_online_endpoint_api_key.png" height="400"/> <br>

1. **Set the Environment Variable:**
   - Add the obtained API key to an environment variable named `AZURE_ML_KEY`.
   - Add the obtained endpoint URI to an environment variable named `AZURE_ML_MANAGED_ENDPOINT`.

## Create a AMLOnlineEndpointChat

After deploying a model and populating your env file, creating an endpoint is as simple as the following

In [1]:

from pyrit.common import default_values

from pyrit.models import ChatMessage
from pyrit.chat import AMLOnlineEndpointChat


default_values.load_default_env()

red_team_chat_engine = AMLOnlineEndpointChat()
red_team_chat_engine.complete_chat(messages=[ChatMessage(role="user", content="Hello world!")])

"Hello! I'm here to help you with any questions you have about data structures and algorithms, coding, or anything else related to computer science. Is there something specific you would like to know or discuss?\n\nHere are a few topics you might be interested in:\n\n* Big O notation and time complexity analysis\n* Common data structures (arrays, linked lists, trees, hash tables, etc.) and when to use them\n* Sorting and searching algorithms\n* Graph algorithms (DFS, BFS, Dijkstra's, etc.)\n* Dynamic programming and greedy algorithms\n* Trees and tree traversal algorithms (pre-order, in-order, post-order)\n* Recursion and stack frames\n\nLet me know if any of these topics interest you, or if you have something else in mind!"


You can then use this cell anywhere you would use a `ChatSupport` object.
For example, you can create a red teaming orchestrator and do the entire [Gandalf Demo](../demo/1_gandalf.ipynb) but use this AML model.
This is also shown in the [Multiturn Demo](../demo/2_multiturn_strategies.ipynb).